In [66]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
import os
import openai
import logging
import sys

from llama_index import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
)
from llama_index.vector_stores import PineconeVectorStore
from llama_index.llms import OpenAI
from llama_index.storage.storage_context import StorageContext


import pinecone

In [68]:
openai.api_key = os.environ["OPENAI_API_KEY"]
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

In [69]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [70]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)

In [71]:
environment = 'gcp-starter'
index_name = "baseline-index"

In [72]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [73]:
# DO NOT RERUN
pinecone.init(api_key=PINECONE_API_KEY)
vector_store = PineconeVectorStore(
    index_name=index_name,
    environment=environment
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

dir_path = 'data'
documents = SimpleDirectoryReader(dir_path).load_data()
vector_index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context,
)
















Upserted vectors: 100%|██████████| 7/7 [00:01<00:00,  4.37it/s]


In [74]:
vector_index.index_struct.index_id = 'patent'
query_engine = vector_index.as_query_engine()

In [77]:
query_engine.query('tell me about data processing')

Response(response='Data processing refers to the manipulation and analysis of data to extract meaningful information and insights. It involves various techniques and methods to transform raw data into a more useful format. In the context provided, the invention described relates to a new and useful method for processing more input streams in the data processing field. The method aims to increase the amount of throughput analyzed by analysis models, such as neural networks, without sacrificing latency, accuracy, or overall system performance. It achieves this by allowing more measurements to be processed using the same processing architecture, without the need for retraining, hyperparameter adjustment, or hardware upgrades. The method involves receiving a measurement set, identifying measurements of interest, selecting measurements to composite, generating composite measurements, and analyzing a batch of measurements. This method enables the processing of a larger volume of data while m